## Generators

In [25]:
def my_func():
    i = 0
    print(i)
    i += 1
    yield "Flying"
    print(i)
    yield 'Jeans'

In [26]:
my_func()

<generator object my_func at 0x7f1504550950>

In [27]:
generator = my_func()
next(generator)

0


'Flying'

In [28]:
next(generator)

1


'Jeans'

In [29]:
from collections.abc import Iterator

isinstance(generator, Iterator)

True

## Making Iterable from Generator

In [30]:
class Sqaures:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return Sqaures.squares_gen(self.n)

    @staticmethod
    def squares_gen(n):
        for i in range(n):
            yield i ** 2

In [31]:
sq = Sqaures(5)

In [32]:
[num for num in sq]

[0, 1, 4, 9, 16]

In [33]:
[num for num in sq]

[0, 1, 4, 9, 16]

## Generator Expressions and Performance

### Generator Expression

In [45]:
g = (i**2 for i in range(10))

In [46]:
type(g)

generator

In [47]:
for item in g:
    print(item)

0
1
4
9
16
25
36
49
64
81


### Performance - Defining and Execution Speed

In [34]:
from timeit import timeit
from math import factorial

def combo(n, k):
    return factorial(n) // (factorial(k) * factorial(n-k))

In [39]:
size = 600
timeit('[[combo(n, k) for k in range(n+1)] for n in range(size+1)]', globals=globals(), number=1)

3.345305541000016

In [55]:
size = 600
timeit('((combo(n, k) for k in range(n+1)) for n in range(size+1))', globals=globals(), number=1)

9.998000678024255e-06

Here, the execution is so fast because generators are lazy and they are just getting defined.  
If we execute both of them then the execution time is almost similar as shown below

In [42]:
def pascal_list(size):
    l = [[combo(n, k) for k in range(n+1)] for n in range(size+1)]
    for row in l:
        for item in row:
            pass

def pascal_gen(size):
    g = ((combo(n, k) for k in range(n+1)) for n in range(size+1))
    for row in g:
        for item in row:
            pass

In [43]:

size = 600
timeit('pascal_list(size)', globals=globals(), number=1)

3.312629384000502

In [44]:
size = 600
timeit('pascal_gen(size)', globals=globals(), number=1)

3.3068681499999

### Performance - Memory

In [48]:
import tracemalloc

In [49]:
def pascal_list(size):
    l = [[combo(n, k) for k in range(n+1)] for n in range(size+1)]
    for row in l:
        for item in row:
            pass
    stats = tracemalloc.take_snapshot().statistics('lineno')
    print(stats[0].size, 'bytes')

In [50]:
def pascal_gen(size):
    g = ((combo(n, k) for k in range(n+1)) for n in range(size+1))
    for row in g:
        for item in row:
            pass
    stats = tracemalloc.take_snapshot().statistics('lineno')
    print(stats[0].size, 'bytes')

In [53]:
tracemalloc.stop()
tracemalloc.clear_traces()
tracemalloc.start()
pascal_list(300)

1998608 bytes


In [54]:
tracemalloc.stop()
tracemalloc.clear_traces()
tracemalloc.start()
pascal_gen(300)

9248 bytes


## Yield From

In [60]:
def matrix(n):
    gen = ( (i * j for j in range(1, n+1))
            for i in range(1, n+1)
          )
    return gen

In [61]:
def matrix_iterator(n):
    for row in matrix(n):
        for item in row:
            yield item

for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


In [63]:
def matrix_iterator(n):
    for row in matrix(n):
        yield from row

for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


## Aggregators

In [64]:
def squares(n):
    for i in range(n):
        yield i**2

In [65]:
min(squares(5))

0

In [66]:
max(squares(5))

16

In [69]:
sum(squares(5))

30

In [70]:
sorted(squares(5), reverse=True)

[16, 9, 4, 1, 0]

## Slicing

In [71]:
import math

def factorials(n):
    for i in range(n):
        yield math.factorial(i)

facts = factorials(100)    

In [72]:
facts[0:2]

TypeError: ignored

In [73]:
def slice_(iterable, start, stop):
    # Exhaust all the values before the starting point
    for _ in range(0, start):
        next(iterable)
        
    # Yield the values in the range of interest
    for _ in range(start, stop):
        yield(next(iterable))

In [74]:
list(slice_(factorials(100), 1, 5))

[1, 2, 6, 24]

In [75]:
from itertools import islice

list(islice(factorials(10), 0, 3))

[1, 1, 2]

In [76]:
list(islice(factorials(10), -1, -3))

ValueError: ignored

## Selecting and Filtering

In [77]:
def gen_cubes(n):
    for i in range(n):
        print(f'yielding {i}')
        yield i**3

def is_odd(x):
    return x % 2 == 1

def is_even(x):
    return x % 2 == 0

### filter()

In [78]:
filtered = filter(is_odd, gen_cubes(10))

### filterfalse()

In [79]:
from itertools import filterfalse

evens = filterfalse(is_odd, gen_cubes(10))

### takewhile()

In [80]:
from itertools import takewhile

list(takewhile(lambda x: 0 <= x <= 0.9, gen_cubes(15)))

yielding 0
yielding 1


[0]

### dropwhile()

In [81]:
from itertools import dropwhile

l = [1, 3, 5, 2, 1]
list(dropwhile(lambda x: x < 5, l))

[5, 2, 1]

### zip()

In [82]:
data = ['a', 'b', 'c', 'd', 'e']
selectors = [True, False, 1, 0]

[item for item, truth_value in zip(data, selectors) if truth_value]

['a', 'c']

### compress()

In [83]:
from itertools import compress
list(compress(data, selectors))

['a', 'c']

## Infinite Iterators

In [84]:
from itertools import (
    count,
    cycle,
    repeat, 
    islice)

### count()

In [85]:
from decimal import Decimal

g = count(10)
g = count(10, step=2)
g = count(1+1j, 1+2j)
g = count(Decimal('0.0'), Decimal('0.1'))

### cycle()

In [86]:
g = cycle(('red', 'green', 'blue'))
list(islice(g, 8))

def colors():
    yield 'red'
    yield 'green'
    yield 'blue'
cols = colors()
g = cycle(cols)

### repeat()

In [89]:
g = repeat('Python')
for _ in range(5):
    print(next(g))

Python
Python
Python
Python
Python
